# Finalizing Poem selections
A quick script to combine final manual evlations and export to a CSV/JSON for ranking and selecting the final collection of limericks for publication.

In [ ]:
import json
import pandas as pd

In [ ]:
# get phil's best
fname = '04_25_poems_phil.json'
with open(fname) as f:
  phils = json.load(f)
misc = [
        23596, 23615, 23625, 23937, 23960, 23465,                                                   #    0 -  199
        25974,                                                                                      #  400 -  599
        34171, 34213,                                                                               # 1600 - 1799
]
decent = [
          23399, 23463, 23490, 23627, 23729, 23737, 23788, 23792, 23813, 23873, 24141, 24183, 24418, #    0 -  199
          24613, 24675, 24925, 25000, 25197, 25258, 25268, 25380, 25644, 25788, 25820,               #  200 -  399
          27509, 27563, 27789, 27831, 27949,                                                         #  600 -  799
          28553, 28915, 28966, 29830,                                                                #  800 -  999
          30059, 30159, 30503, 30778,                                                                # 1000 - 1199
          34021, 34088, 34093, 34217, 34264, 34352, 34421, 34465,                                    # 1600 - 1799
]
great = [
         23429, 23526, 23544, 23775, 23950, 24062, 24126, 24127, 24179, 24321, 24464, 24532,        #    0 -  199
         24595, 24807, 25444, 25466, 25603, 25724, 25747, 25765,                                    #  200 -  399
         25966, 25987, 26229, 26452, 26934,                                                         #  400 -  599
         28297, 28342,                                                                              #  600 -  799
         34131, 34460,                                                                              # 1600 - 1799
]
for k, v in phils.items():
  int_k = int(k.split('04_20_')[1])
  v['reviewer'] = 'phil'
  if int_k in misc:
    v['score'] = 3
  elif int_k in decent:
    v['score'] = 2
  elif int_k in great:
    v['score'] = 1
  else:
    v['score'] = None

# phils_great = {f"04_20_{id}": phils[f"04_20_{id}"] for id in phils_great}
# for k, v in phils_great.items():
#   v['reviewer'] = 'phil'
phil_counts = [0, 0, 0]
for k, v in phils.items():
  if v['score']:
    phil_counts[v['score']-1] += 1
print(sum(phil_counts))
phil_counts = dict(zip(['great', 'decent', 'misc'], phil_counts))
print(phil_counts)

83
{'great': 29, 'decent': 45, 'misc': 9}


In [ ]:
# get kai's best
fname = '04_25_poems_kai_evaluated.json'
with open(fname) as f:
  kais = json.load(f)
for k, v in kais.items():
  v['reviewer'] = 'kai'
print(len(kais))
kais_counts = [0, 0, 0]
for k, v in kais.items():
  if v['score']:
    kais_counts[v['score']-1] += 1
kais_counts = dict(zip(['great', 'decent', 'misc'], kais_counts))
print(kais_counts)

312
{'great': 57, 'decent': 133, 'misc': 122}


In [ ]:
# get rami's best
fname = '04_25_poems_rami_evaluated.json'
with open(fname) as f:
  ramis = json.load(f)
# ramis_best = {k: v for k, v in ramis.items() if v['score']==1}
for k, v in ramis.items():
  v['reviewer'] = 'rami'
print(len(ramis))
ramis_counts = [0, 0, 0]
for k, v in ramis.items():
  if v['score']:
    ramis_counts[v['score']-1] += 1
ramis_counts = dict(zip(['great', 'decent', 'misc'], ramis_counts))
print(ramis_counts)

536
{'great': 47, 'decent': 89, 'misc': 400}


In [ ]:
# Combine ranked poems into one dict
ranked_poems = {}
for k, v in phils.items():
  if v['score']:
    ranked_poems[k] = v
ranked_poems.update(kais)
ranked_poems.update(ramis)
len(ranked_poems)

931

In [ ]:
# Export all manual evaluated poems into a single JSON
with open('04_25_all_evaluated.json', 'w') as f:
  json.dump(ranked_poems, f)

In [ ]:
# package into dataframe
unpack_classes = {k: v['class'] for k, v in ranked_poems.items()}  # pull out classes into own dict for flattening
ranked_poems_df = pd.concat([pd.DataFrame.from_dict(ranked_poems, orient='index').drop(['class'], axis=1), 
                             pd.DataFrame.from_dict(unpack_classes).T.astype('float64')], axis=1)

In [ ]:
# Check categories counts over ranked_poems
ranked_poems_df[ranked_poems_df['score'] == 1].describe().iloc[0, 5:].sort_values(ascending=False)[:20]

/Arts & Entertainment                              41.0
/People & Society/Religion & Belief                 8.0
/Health                                             7.0
/Health/Health Conditions                           6.0
/Books & Literature                                 6.0
/Arts & Entertainment/Music & Audio                 6.0
/Food & Drink/Food                                  5.0
/Books & Literature/Poetry                          5.0
/People & Society                                   5.0
/Hobbies & Leisure                                  4.0
/Sensitive Subjects                                 4.0
/Finance/Investing                                  2.0
/Pets & Animals/Pets/Dogs                           2.0
/Science/Biological Sciences                        2.0
/Arts & Entertainment/Humor                         2.0
/News/Politics                                      2.0
/Reference/Language Resources                       2.0
/Science                                        

In [ ]:
# Export ranked poems to CSV for final evaluation
ranked_poems_df.to_csv('04_25_all_evaluated.csv')

In [ ]:
# Check oedilf's rankings
fname = 'oedilf_ld_class_72432.json'
with open(fname) as f:
  oedilf = json.load(f)